In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import glob as gb
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model

import warnings
import matplotlib.pyplot as plt

sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
arabic_alphabet = ['ع', 'ال', 'أ', 'ب', 'د', 'ظ', 'ض', 'ف', 'ق', 'غ', 'ه', 'ح', 'ج', 'ك', 'خ', 'لا', 'ل',
                   'م', 'ن', 'ر', 'ص', 'س', 'ش', 'ط', 'ت', 'ث', 'ذ', 'ة', 'و', 'ئ', 'ي', 'ز',' ']


In [ ]:
data = pd.read_csv(r'C:\Users\mosta\Downloads\Telegram Desktop\fvf\ArASL_Database_54K\Labels\ImagesClassPath.csv')
data.head(-5)

,id,ClassId,ImagePath
0,1,0,ArASL_Database_54K/ain/AIN (1).JPG
1,2,0,ArASL_Database_54K/ain/AIN (10).JPG
2,3,0,ArASL_Database_54K/ain/AIN (100).jpg
3,4,0,ArASL_Database_54K/ain/AIN (1000).JPG
4,5,0,ArASL_Database_54K/ain/AIN (1001).JPG
...,...,...,...
57039,57040,32,ArASL_Database_54K/space/del2991.jpg
57040,57041,32,ArASL_Database_54K/space/del2992.jpg
57041,57042,32,ArASL_Database_54K/space/del2993.jpg
57042,57043,32,ArASL_Database_54K/space/del2994.jpg


In [ ]:
for i in range(data.shape[0]):
    data.loc[i, 'ClassId'] = arabic_alphabet[data.loc[i, 'ClassId']]
data.head(-5)

,id,ClassId,ImagePath
0,1,ع,ArASL_Database_54K/ain/AIN (1).JPG
1,2,ع,ArASL_Database_54K/ain/AIN (10).JPG
2,3,ع,ArASL_Database_54K/ain/AIN (100).jpg
3,4,ع,ArASL_Database_54K/ain/AIN (1000).JPG
4,5,ع,ArASL_Database_54K/ain/AIN (1001).JPG
...,...,...,...
57039,57040,,ArASL_Database_54K/space/del2991.jpg
57040,57041,,ArASL_Database_54K/space/del2992.jpg
57041,57042,,ArASL_Database_54K/space/del2993.jpg
57042,57043,,ArASL_Database_54K/space/del2994.jpg


In [ ]:
df = pd.DataFrame({'img':data['ImagePath'],
                   'class':data['ClassId']})
df.head(-5)

,img,class
0,ArASL_Database_54K/ain/AIN (1).JPG,ع
1,ArASL_Database_54K/ain/AIN (10).JPG,ع
2,ArASL_Database_54K/ain/AIN (100).jpg,ع
3,ArASL_Database_54K/ain/AIN (1000).JPG,ع
4,ArASL_Database_54K/ain/AIN (1001).JPG,ع
...,...,...
57039,ArASL_Database_54K/space/del2991.jpg,
57040,ArASL_Database_54K/space/del2992.jpg,
57041,ArASL_Database_54K/space/del2993.jpg,
57042,ArASL_Database_54K/space/del2994.jpg,


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=True)


print('Train shape:', df_train.shape)
print('Test shape:', df_test.shape)

Train shape: (51344, 2)
Test shape: (5705, 2)


In [ ]:
df_train

,img,class
50914,ArASL_Database_54K/ya/YA (578).JPG,ئ
53919,ArASL_Database_54K/zay/ZAY (882).jpg,ز
55848,ArASL_Database_54K/space/del1800.jpg,
26132,ArASL_Database_54K/la/LA (1446).jpg,لا
55217,ArASL_Database_54K/space/del1169.jpg,
...,...,...
23766,ArASL_Database_54K/kaaf/KAAF (763).JPG,ك
53666,ArASL_Database_54K/zay/ZAY (654).jpg,ز
43003,ArASL_Database_54K/taa/TAA (867).jpg,ت
13411,ArASL_Database_54K/fa/FA (557).jpg,ف


In [ ]:
size  = 64
channels = 1
batch = 128
epochs = 25
steps_per_epoch=  data.shape[0] // batch

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             zoom_range=0.2,
                             width_shift_range=.2, height_shift_range=.2,
                             rotation_range=30,
                             brightness_range=[0.8, 1.2],
                             horizontal_flip=False)

datagenScale = ImageDataGenerator(rescale=1./255)

In [ ]:
train_path =r'C:\Users\mosta\Downloads\Telegram Desktop\fvf' 


In [ ]:
X_train = datagen.flow_from_dataframe(
    df_train, 
    directory = train_path, 
    x_col = 'img',
    y_col = 'class',
    target_size = (size,size),
    color_mode = 'grayscale',
    shuffle = True,
    batch_size = batch)

X_test = datagenScale.flow_from_dataframe(
    df_test, 
    directory = train_path, 
    x_col = 'img',
    y_col = 'class',
    target_size= (size,size),
    color_mode = 'grayscale',
    shuffle = True,
    batch_size = batch)

Found 51343 validated image filenames belonging to 33 classes.
Found 5705 validated image filenames belonging to 33 classes.


In [ ]:
X_train.class_indices

{' ': 0,
 'أ': 1,
 'ئ': 2,
 'ال': 3,
 'ب': 4,
 'ة': 5,
 'ت': 6,
 'ث': 7,
 'ج': 8,
 'ح': 9,
 'خ': 10,
 'د': 11,
 'ذ': 12,
 'ر': 13,
 'ز': 14,
 'س': 15,
 'ش': 16,
 'ص': 17,
 'ض': 18,
 'ط': 19,
 'ظ': 20,
 'ع': 21,
 'غ': 22,
 'ف': 23,
 'ق': 24,
 'ك': 25,
 'ل': 26,
 'لا': 27,
 'م': 28,
 'ن': 29,
 'ه': 30,
 'و': 31,
 'ي': 32}

In [ ]:
optimizer = Adam(learning_rate=0.001)
Model = Sequential([Conv2D(filters=33,  kernel_size=(3,3), activation="relu", input_shape=(size,size,channels)),
                    BatchNormalization(),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.25),
                    
                    Conv2D(filters=128,  kernel_size=(3,3), activation="relu"),
                    BatchNormalization(),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.25),
                 
                    Conv2D(filters=512,  kernel_size=(3,3), activation="relu"),
                    BatchNormalization(),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.25),
                    
                    Conv2D(filters=2048,  kernel_size=(3,3), activation="relu"),
                    BatchNormalization(),
                    MaxPool2D(2,2, padding='same'),
                    Dropout(0.25),
                    
                    
                    Flatten(),
                    BatchNormalization(),

                    
                    Dense(units=4096, activation="relu"), 
                    BatchNormalization(),
                    Dropout(0.25),
                    
                                        
                    Dense(units=1024, activation="relu"), 
                    BatchNormalization(),
                    Dropout(0.25),
                                  
                                  
                    Dense(units=256, activation="relu"),
                    BatchNormalization(),
                    Dropout(0.25),
                    
                                                      
                    Dense(units=64, activation="relu"),
                    BatchNormalization(),
                    Dropout(0.5),
                    
                    
                    Dense(units=33, activation="softmax"),
])

Model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])


Model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 33)        330       
                                                                 
 batch_normalization_18 (Bat  (None, 62, 62, 33)       132       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 31, 31, 33)       0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 31, 31, 33)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 29, 29, 128)       38144     
                                                                 
 batch_normalization_19 (Bat  (None, 29, 29, 128)     

In [ ]:
import keras
from keras.callbacks import EarlyStopping
import tensorflow as tf
# Define early stopping criteria
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min')

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint('arabic.h5',verbose=1,save_best_only=True)

In [ ]:
history = Model.fit(X_train, validation_data=X_test, epochs=epochs,  callbacks=[early_stopping, checkpointer])

Epoch 1/25
402/402 [==============================] - ETA: 0s - loss: 3.0567 - accuracy: 0.1758
Epoch 1: val_loss improved from inf to 3.74102, saving model to arabic.h5
402/402 [==============================] - 1711s 4s/step - loss: 3.0567 - accuracy: 0.1758 - val_loss: 3.7410 - val_accuracy: 0.0601
Epoch 2/25
402/402 [==============================] - ETA: 0s - loss: 1.6377 - accuracy: 0.4866
Epoch 2: val_loss did not improve from 3.74102
402/402 [==============================] - 1622s 4s/step - loss: 1.6377 - accuracy: 0.4866 - val_loss: 6.2590 - val_accuracy: 0.1104
Epoch 3/25
402/402 [==============================] - ETA: 0s - loss: 1.0494 - accuracy: 0.6687
Epoch 3: val_loss improved from 3.74102 to 0.96381, saving model to arabic.h5
402/402 [==============================] - 1589s 4s/step - loss: 1.0494 - accuracy: 0.6687 - val_loss: 0.9638 - val_accuracy: 0.6729
Epoch 4/25
402/402 [==============================] - ETA: 0s - loss: 0.8014 - accuracy: 0.7471
Epoch 4: val_loss 

In [ ]:
from tensorflow.keras.models import load_model

Model = load_model('C:/Users/mosta/Downloads/Telegram Desktop/fvf/lastarabi.h5')


In [ ]:
Model.save('lastarabi.h5')

test from here


In [4]:
class_names = {' ': 0,'أ': 1,'ئ': 2,
 'ال': 3,
 'ب': 4,
 'ة': 5,
 'ت': 6,
 'ث': 7,
 'ج': 8,
 'ح': 9,
 'خ': 10,
 'د': 11,
 'ذ': 12,
 'ر': 13,
 'ز': 14,
 'س': 15,
 'ش': 16,
 'ص': 17,
 'ض': 18,
 'ط': 19,
 'ظ': 20,
 'ع': 21,
 'غ': 22,
 'ف': 23,
 'ق': 24,
 'ك': 25,
 'ل': 26,
 'لا': 27,
 'م': 28,
 'ن': 29,
 'ه': 30,
 'و': 31,
 'ي': 32}

In [26]:
score  = Model.evaluate(X_test)
print("Loss :",score[0])
print("Accuracy :",score [1])

45/45 [==============================] - 19s 416ms/step - loss: 0.4027 - accuracy: 0.9050
Loss : 0.40269216895103455
Accuracy : 0.9049956202507019


In [36]:
# Import necessary libraries
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img1 = load_img(r'C:/Users/mosta/OneDrive/الصور/لفة كاميرا/WIN_20230501_22_03_58_Pro.jpg', target_size=(64, 64),color_mode = 'grayscale')
img1 = img_to_array(img1)
img1 = np.expand_dims(img1, axis=1)
img1 = img1 / 255.0
img1 = np.squeeze(img1, axis=1)


img2 = load_img(r'C:/Users/mosta/OneDrive/الصور/لفة كاميرا/WIN_20230501_22_04_00_Pro.jpg', target_size=(64, 64),color_mode = 'grayscale')
img2 = img_to_array(img2)
img2 = np.expand_dims(img2, axis=1)
img2 = img2 / 255.0
img2 = np.squeeze(img2, axis=1)

img3 = load_img(r'C:/Users/mosta/OneDrive/الصور/لفة كاميرا/WIN_20230501_22_08_32_Pro.jpg', target_size=(64, 64),color_mode = 'grayscale')
img3 = img_to_array(img3)
img3 = np.expand_dims(img3, axis=1)
img3 = img3 / 255.0
img3 = np.squeeze(img3, axis=1)


In [37]:
predictions = Model.predict(np.array([img1,img2,img3]))
predictions

1/1 [==============================] - 0s 53ms/step


array([[9.19321974e-06, 9.95611012e-01, 3.05412215e-07, 9.48136403e-06,
        1.09178181e-04, 3.53931318e-06, 1.71615411e-05, 5.73261332e-06,
        1.12292480e-06, 1.95017992e-06, 2.19122976e-05, 1.73667911e-06,
        5.90254103e-07, 1.12830567e-05, 2.51111351e-06, 8.04032752e-05,
        6.21207937e-06, 1.25212080e-06, 1.00204352e-06, 9.75284274e-06,
        2.99786243e-05, 9.98278774e-05, 3.62555147e-03, 2.67436167e-06,
        1.46145453e-06, 1.35613591e-04, 4.67360042e-06, 1.10271037e-06,
        4.03958302e-06, 7.09592632e-06, 2.46924770e-07, 1.14482418e-05,
        1.70915460e-04],
       [1.49237749e-07, 1.79872998e-07, 1.71738765e-08, 8.33524439e-07,
        1.48968848e-09, 1.66121472e-09, 5.62424596e-09, 3.16399763e-07,
        1.59087836e-06, 6.60756001e-08, 9.39040756e-09, 1.42367396e-09,
        2.06833519e-08, 3.86706750e-10, 1.42459005e-08, 9.99931931e-01,
        8.40212124e-06, 8.53408892e-07, 1.57317118e-07, 8.14505477e-11,
        1.86381111e-09, 7.72672593e-09,

In [38]:
t =[]
for i in range(0,len(predictions)):
    s=np.argmax(predictions[i])
    t.append(s)
print(t)


[1, 15, 32]


In [39]:
word = np.zeros(len(t), dtype=str)
for key,value in class_names.items():
    for i in range(0,len(t)):
        if t[i] == value:
            word[i]=key
word       
sent= ''
for i in range(0,len(t)):
    sent+=word[i]
print(sent)

أسي
